In [1]:
import pandas
import root_numpy
import numpy

In [2]:
channel = 'Bu_JPsiK'

In [3]:
data = pandas.read_csv('../datasets/data/csv/WG/{}/2012/Tracks.csv'.format(channel), sep='\t')

/mnt/mfs/miniconda/envs/rep_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data = data.query('(Bmass < 5.4) & (Bmass > 5.16)')
data.loc[~numpy.isfinite(data['IPs'].values), 'IPs'] = -999
data.loc[~numpy.isfinite(data['IPerr'].values), 'IPerr'] = -999

In [7]:
event_id = data.run.apply(str) + '_' + data.event.apply(int).apply(str)
data['event_id'] = event_id

In [10]:
splot = pandas.read_csv('../datasets/data/csv/WG/{}/2012/sPlot_{}.csv'.format(channel, channel),
                        sep=' ', header=None).drop([8], axis=1)
splot.columns = ['run', 'evTop', 'evBottom', 'Bmass', 'Nbkg_tot_sw', 'L_Nbkg_tot', 'Nsig_tot_sw', 'L_Nsig_tot']
event_id_column = 'event_id'

splot[event_id_column] = splot.run.apply(str) + '_' + (splot.evTop * 1e6 + splot.evBottom).apply(int).apply(str)
index = numpy.unique(splot[event_id_column], return_index=True)[1]
splot.index = numpy.arange(len(splot))
splot = splot.ix[index, :]

In [11]:
splot.index = splot[event_id_column]
data.index = data[event_id_column]

splot.index = splot[event_id_column]
data.index = data[event_id_column]

start = 0
end = 1000000
sweight = []
for batch in range(33):
    if end > len(data):
        end = len(data)
    sweight.append(splot.loc[data.index[start:end], 'Nsig_tot_sw'].values)
    start = end
    end += 1000000
    
sweight = numpy.concatenate(sweight)
data.N_sig_sw = sweight

In [12]:
data = data.drop('event_id', axis=1)

In [14]:
import sys
sys.path.insert(0, "../")

In [15]:
from utils import shrink_floats

In [16]:
shrink_floats(data)

In [17]:
root_numpy.array2root(data.to_records(index=False), '../datasets/data/csv/WG/{}/2012/Tracks.root'.format(channel),
                      mode='recreate')

/mnt/mfs/miniconda/envs/rep_py2/lib/python2.7/site-packages/root_numpy/_tree.py:449: UserWarning: converter for dtype('O') is not implemented (skipping)
  _librootnumpy.array2root(arr, filename, treename, mode)


In [18]:
data.head()

,run,event,Bmass,time,i,mult,partP,partPt,ptB,IPs,...,Dist_phi,N_sig_sw,mu_cut,e_cut,K_cut,MCID,OS_SS,xFlag,K_MCID,BOosc
event_id,,,,,,,,,,,,,,,,,,,,,
132623_227572381,132623,227572384.0,5.280417,2.739783,0,16,2.975773,0.125786,6.62858,0.346570,...,1.714190,1.12428,0,0,0,-0,0.0,-0.000563,0,-1878611067
132623_227572381,132623,227572384.0,5.280417,2.739783,1,16,2.113523,0.626089,6.62858,1.647396,...,0.118375,1.12428,0,0,0,-0,0.0,1.000000,0,-1878611067
132623_227572381,132623,227572384.0,5.280417,2.739783,2,16,3.391397,0.237943,6.62858,2.565533,...,2.131909,1.12428,0,0,0,-1.63e-05,0.0,1.000000,0,-1878611067
132623_227572381,132623,227572384.0,5.280417,2.739783,3,16,12.321827,0.512020,6.62858,2.054170,...,1.768889,1.12428,0,0,0,7e-08,0.0,0.000000,0,-1878611067
132623_227572381,132623,227572384.0,5.280417,2.739783,4,16,47.236053,1.710158,6.62858,0.995251,...,1.990477,1.12428,0,0,0,-9.90309e+16,0.0,-0.000000,0,-1878611067
